In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

In [85]:
[(s,v.stoi[s]) for s in tst]

[('xxunk', 0),
 ('xxpad', 1),
 ('xxbos', 2),
 ('xxfld', 3),
 ('xxmaj', 4),
 ('xxup', 5),
 ('xxrep', 6),
 ('xxwrep', 7)]

In [3]:
PATH=Path('data/wikitext-2')

In [4]:
# download_url('https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip', PATH)

In [5]:
def read_file(fn:PathOrStr, enc='utf-8'):
    "Read the text in `fn`."
#     with open(fn,'r', encoding = enc) as f: return ''.join(f.read().splitlines())
    tokens = []
    with open(fn,'r', encoding = enc) as f: 
        for line in f.read().splitlines():
            l = line.strip()
            if len(l) == 0: continue
            tokens.append(l)#.split())
    return np.array(tokens)

In [6]:
from fastai.text import data

In [7]:
train_tok = read_file(PATH/'wiki.train.tokens')
valid_tok = read_file(PATH/'wiki.valid.tokens')
test_tok = read_file(PATH/'wiki.test.tokens')

In [8]:
path = PATH

In [9]:
tprer = defaults.text_pre_rules

In [10]:
tpostr = defaults.text_post_rules

In [11]:
tst = defaults.text_spec_tok

In [12]:
def fix_html(x:str) -> str:
    "List of replacements from html strings in `x`."
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
#         'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
#         '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
#     return x
    return re1.sub(' ', html.unescape(x))

In [34]:
def fix_html(x:str) -> str:
    "List of replacements from html strings in `x`."
    re1 = re.compile(r'  +')
    x = x.replace(#'#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
#         'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
#         '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-')#.replace('\\', ' \\ ')
#     return x
    return re1.sub(' ', html.unescape(x))

In [57]:
def fix_html(x:str) -> str:
    "List of replacements from html strings in `x`."
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
#         'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
#         '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-')#.replace('\\', ' \\ ')
#     return x
    return re1.sub(' ', html.unescape(x))

In [72]:
def fix_html(x:str) -> str:
    "List of replacements from html strings in `x`."
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
#         'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
#         '<br />', "\n").replace('\\"', '"').replace(
        '<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-')#.replace('\\', ' \\ ')
#     return x
    return re1.sub(' ', html.unescape(x))
fail

In [88]:
def fix_html(x:str) -> str:
    "List of replacements from html strings in `x`."
    re1 = re.compile(r'  +')
    x = x.replace(#'#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
#         'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
#         '<br />', "\n").replace('\\"', '"').replace(
#         '<unk>',UNK).replace(#' @.@ ','.').replace(
        ' @-@ ','-')#.replace('\\', ' \\ ')
#     return x
    return re1.sub(' ', html.unescape(x))

# replacing unk fails

In [18]:
UNK

'xxunk'

In [59]:
tst_2 = ['xxunkxx'] + tst

In [60]:
tst_2

['xxunkxx',
 'xxunk',
 'xxpad',
 'xxbos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [62]:
def fix_html(x:str) -> str:
    return x.replace('<unk>', 'xxunkxx')

# replacing unk fails

In [63]:
# def fix_html(x:str) -> str:
#     "List of replacements from html strings in `x`."
#     re1 = re.compile(r'  +')
#     x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
#         'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
#         '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
#         ' @-@ ','-').replace('\\', ' \\ ')
# #     return x
#     return re1.sub(' ', html.unescape(x))

In [64]:
fix_html('dsfdsdjfkldss <unk>')

'dsfdsdjfkldss xxunkxx'

In [65]:
tprer = [fix_html] + tprer[1:]

In [66]:
tok = Tokenizer(pre_rules=tprer, post_rules=tpostr, special_cases=tst_2)

In [67]:
# tok.pre_rules  = []
# tok.post_rules = []
# tok.special_cases = []

In [76]:
processor = data._get_processor(tokenizer=tok, vocab=None, max_vocab=10)
src = ItemLists(path, TextList(train_tok, path=path, processor=processor),
                TextList(valid_tok, path=path, processor=processor))
src = src.label_for_lm()
if test_tok is not None: src.add_test(TextList(test_tok, path=path))
db = src.databunch()

In [81]:
db.save('tmp_tok_28')

In [77]:
for i,(x,y) in enumerate(db.train_dl):
    print(x.shape, x.max(), x.float().mean(), x.type())
    print(y.shape, y.max(), y.float().mean(), y.type())
    if i > 6: break

torch.Size([64, 95]) tensor(16, device='cuda:0') tensor(3.3507, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 95]) tensor(16, device='cuda:0') tensor(3.3260, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 70]) tensor(16, device='cuda:0') tensor(3.2942, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 70]) tensor(16, device='cuda:0') tensor(3.2964, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 71]) tensor(16, device='cuda:0') tensor(3.2044, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 71]) tensor(16, device='cuda:0') tensor(3.2053, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 77]) tensor(16, device='cuda:0') tensor(3.3164, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 77]) tensor(16, device='cuda:0') tensor(3.3202, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 71]) tensor(16, device='cuda:0') tensor(3.4140, device='cuda:0') torch.cuda.LongTensor
torch.Size([64, 71]) tensor(16, device='cuda:0') tensor(3.4151, device='c

In [78]:
len(db.vocab.itos)

17

In [79]:
v = db.train_ds.vocab

In [74]:
v.stoi['xxunkxx']

12

In [24]:
db.train_ds.vocab.

SyntaxError: invalid syntax (<ipython-input-24-6eed0a09f19c>, line 1)

In [19]:
ob[0].max()

tensor(29060)

In [14]:
ob = db.one_batch(); ob

(tensor([[   2,    4,    8,  ..., 4780,    9,   38],
         [   4, 1001,    4,  ...,   26,    4, 3050],
         [ 199,  300,    9,  ...,   86,    9,   19],
         ...,
         [   4,   44,   19,  ...,   15,  458,   13],
         [2847,  557,  205,  ...,   28,   12, 2076],
         [   4,    8,  566,  ...,    4,  679,  141]]),
 tensor([[    4,     8,  6936,  ...,     9,    38,  1391],
         [ 1001,     4,   229,  ...,     4,  3050,    20],
         [  300,     9,   358,  ...,     9,    19,    12],
         ...,
         [   44,    19,     8,  ...,   458,    13,     0],
         [  557,   205,  7993,  ...,    12,  2076, 15853],
         [    8,   566,   549,  ...,   679,   141,     8]]))

In [80]:
db.show_batch()

idx,text
0,"xxbos xxunk xxunk xxunk xxmaj xxunk to xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxbos xxmaj xxunk xxunk xxunk xxunk xxunk , xxunk xxunk xxmaj xxunk xxmaj xxunk xxmaj xxunk xxunk xxunk to xxunk , xxunk xxunk xxunk xxunk xxunkxx xxunk xxmaj xxunk xxunkxx xxunk xxmaj xxunk xxunkxx , xxmaj xxunk . xxmaj xxunk and xxmaj xxunk xxunk in xxmaj xxunk xxunk xxmaj xxunk xxunk , xxunk and xxunk xxunk"
1,"xxmaj xxunk xxunk xxunk xxunk xxbos xxmaj the xxunk xxunk xxunk xxunk the xxunk of xxunk xxunk xxunk xxmaj xxunk and xxunk xxunk xxunk xxunk xxunk xxunkxx . xxmaj xxunk xxunk xxunk xxunk xxunk a xxunk xxunk , xxunk xxunk xxunk xxunk in xxmaj xxunk . xxmaj in xxmaj xxunk xxunk xxunk , xxunk xxunk xxunk of the xxunk xxunk xxunk xxunk xxunk xxunk to xxunk xxunk and xxunk xxunk"
2,"xxunk xxunk xxunk xxunk xxunk in the xxunk , xxunk xxunkxx xxunk in xxunk and xxunk xxunk xxunk xxunk xxunk to xxunk the xxunk xxunk . xxbos xxmaj the xxmaj xxunk xxunk xxunk xxunk in the xxunk of xxmaj xxunk xxunk in xxmaj xxunk xxunk xxunk xxunk of the xxunk xxunkxx xxunk xxunk xxunk . xxmaj xxunk , the xxunk of xxmaj xxunk xxunk xxunk xxunk xxunk xxmaj xxunk ."
3,"a xxunk to xxunk the xxunk xxunk xxunk , xxunk xxunk xxunk xxunk the xxunk xxunk xxunk the xxunk . xxmaj the xxunk xxunkxx xxunk to a xxmaj xxunk xxmaj xxunk xxunk xxunk xxmaj xxunk and xxmaj xxunk xxunk the xxunk . xxmaj xxunk xxmaj xxunk xxunk the xxunk , xxunk xxunk xxunk xxunk xxunk , xxunk the xxunk xxunk xxunk xxunk to xxunk . xxmaj the xxunk xxunk xxmaj"
4,", xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk in xxunk . xxmaj xxunk xxunk xxmaj xxunk xxmaj xxunk xxunk xxunk xxunk xxunk xxmaj xxunk and xxmaj xxunk xxunk xxunk , the xxunk xxunk xxunk in the xxunk xxunk , and xxunk xxunk xxunk xxunk xxunk a xxunk xxunk xxunk . xxmaj xxunk xxmaj xxunk xxunk , xxunk , xxmaj xxunk xxunk xxunk to xxunk the xxunk , xxunk"


In [15]:
len(train_tok), len(valid_tok), len(test_tok)

(1165029, 2461, 2891)

In [22]:
train_tok[:5][:5]

array([list(['=', 'Valkyria', 'Chronicles', 'III', '=']),
       list(['Senjō', 'no', 'Valkyria', '3', ':', '<unk>', 'Chronicles', '(', 'Japanese', ':', '戦場のヴァルキュリア3', ',', 'lit', '.', 'Valkyria', 'of', 'the', 'Battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'Valkyria', 'Chronicles', 'III', 'outside', 'Japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', 'Sega', 'and', 'Media.Vision', 'for', 'the', 'PlayStation', 'Portable', '.', 'Released', 'in', 'January', '2011', 'in', 'Japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'Valkyria', 'series', '.', 'Employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', 'predecessors', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', '"', 'Nameless', '"', ',', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'Gallia', 'during', 'the', 'Second', 'Europan', 'War'

In [17]:
text_data = TextLMDataBunch.from_tokens(
    PATH, train_tok, valid_tok, test_tok, None, None)

In [18]:
vocab = text_data.train_ds.vocab

In [19]:
text_data.save()

### Wikitext-2

In [7]:
PATH=Path('data/wikitext-2')

In [ ]:
# download_url('https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip', PATH)

In [9]:
train_tok = read_file(PATH/'wiki.train.tokens')
valid_tok = read_file(PATH/'wiki.valid.tokens')
test_tok = read_file(PATH/'wiki.test.tokens')

In [ ]:
train_tok[:100]

In [8]:
len(train_tok), len(valid_tok), len(test_tok)

(36718, 3760, 4358)

In [9]:
' '.join(train_tok[4][:20])

'The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II'

In [10]:
text_data = TextLMDataBunch.from_tokens(
    PATH, train_tok, valid_tok, test_tok, None, None)

In [11]:
text_data.save()

## Loading data

In [ ]:
data = TextLMDataBunch.load(PATH)